In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sys
sys.path.append("/Users/mahdi.sheikholeslami/REPOS/discrete-representation/jax/pure_jax")

In [365]:
import jax.numpy as jnp
from jax import grad, jit, vmap, random
import numpy as np
import optax
import jax

In [366]:
from model import mlp_init_network_params, mlp_predict, mlp_random_layer_params, mlp_representation
from datautils import get_datasets, NumpyLoader, mnist_dataset, mnist_dataset_test
from utils import keyGen
from train import fit, loss_optax, accuracy

In [367]:
mnist_test_image = jnp.concatenate([mnist_dataset_test[i][0][None, :] for i in range(1000)], axis = 0)
mnist_test_labels = jnp.array([mnist_dataset_test[i][1] for i in range(1000)])[:, None]

In [368]:
mnist_test_labels.shape

(1000, 1)

In [369]:
jnp.concatenate( [ jnp.array([[2,3]]), jnp.array([[4,5]]) ])

DeviceArray([[2, 3],
             [4, 5]], dtype=int32)

In [370]:
keygen = keyGen()

In [371]:
mlp_predict(mlp_params, next(b)[0], keygen())

DeviceArray([[-0.13150859,  0.04672344, -0.04399896,  0.01546367,
               0.03607774,  0.08812708, -0.05662421,  0.00067977,
              -0.08499786, -0.03657673],
             [-0.15354598,  0.03896669, -0.04109894,  0.04463703,
               0.03662867,  0.0779243 , -0.05236621,  0.03147275,
              -0.08654519, -0.05707055]], dtype=float32)

In [372]:
sizes = [784, 500, 200, 50, 10]

In [373]:
mlp_params = mlp_init_network_params(sizes, keygen())

In [374]:
optimizer = optax.adam(learning_rate=1e-4)

In [375]:
DATAGEN = NumpyLoader(mnist_dataset, batch_size=50)

In [376]:
mlp_predict_ = lambda params, image: mlp_predict(params, image, 0, noise_status=False)
loss_func = lambda params, image, targets, rkey : loss_optax(params, image, targets, mlp_predict, rkey, noise_status=False)   
acc_func = lambda params, images, targets: accuracy(params, images, targets, mlp_predict_)

In [377]:
pps = jnp.zeros([4, 10])

In [378]:
mnist_test_labels[:4]

DeviceArray([[7],
             [2],
             [1],
             [0]], dtype=int32)

In [379]:
pps = pps.at[jnp.arange(4), mnist_test_labels[:4].squeeze()].set(100)

In [380]:
for epoch in range(5):
    mlp_params = fit(mlp_params, optimizer, DATAGEN, keygen, loss_func)
    print(f"acc: {acc_func(mlp_params, mnist_test_image, mnist_test_labels)} ")

step 0, loss: 2.3010683059692383
step 100, loss: 2.044616937637329
step 200, loss: 1.8653136491775513
step 300, loss: 1.6460514068603516
step 400, loss: 1.5417754650115967
step 500, loss: 1.323885202407837
step 600, loss: 1.2946404218673706
step 700, loss: 1.0740212202072144
step 800, loss: 1.0058300495147705
step 900, loss: 0.9718137979507446
step 1000, loss: 0.8122867941856384
step 1100, loss: 0.7607800364494324
acc: 0.9510000348091125 
step 0, loss: 0.7227511405944824
step 100, loss: 0.6341555118560791
step 200, loss: 0.6088122129440308
step 300, loss: 0.4648611843585968
step 400, loss: 0.6009835600852966
step 500, loss: 0.335225373506546
step 600, loss: 0.4709201753139496
step 700, loss: 0.2967621386051178
step 800, loss: 0.32448774576187134
step 900, loss: 0.3433608114719391
step 1000, loss: 0.23497577011585236
step 1100, loss: 0.24862051010131836
acc: 0.9630000591278076 
step 0, loss: 0.2538604438304901
step 100, loss: 0.25118499994277954
step 200, loss: 0.30953270196914673
step 

In [381]:
mlp_representation(mlp_params, mnist_test_image[0])

DeviceArray([1.00000000e+00, 3.18334729e-04, 1.00000000e+00,
             9.92960274e-01, 9.99999404e-01, 9.99999404e-01,
             9.99998927e-01, 9.91979277e-07, 9.99997735e-01,
             9.99998808e-01, 1.00000000e+00, 1.94914729e-09,
             1.20089494e-08, 1.00000000e+00, 3.28520059e-06,
             1.87737932e-08, 1.29687990e-07, 9.99961972e-01,
             3.85462045e-06, 6.06926287e-09, 9.99988794e-01,
             9.99989629e-01, 4.21799884e-09, 9.99580920e-01,
             1.00000000e+00, 2.64349254e-08, 1.00000000e+00,
             4.59628851e-08, 9.99962687e-01, 9.99992490e-01,
             9.99995708e-01, 9.99994397e-01, 9.99984384e-01,
             9.99993205e-01, 9.10374729e-06, 9.99973416e-01,
             9.99997854e-01, 9.99999881e-01, 5.85319356e-07,
             1.35594689e-06, 1.00000000e+00, 3.62419087e-06,
             1.80771701e-06, 9.99335110e-01, 1.09840464e-07,
             3.47290443e-05, 4.15709678e-07, 9.99710858e-01,
             9.99955893e

In [382]:
jax.nn.sigmoid(1)

DeviceArray(0.7310586, dtype=float32, weak_type=True)